<a href="https://colab.research.google.com/github/ShwetaAroraKhera/ShwetaAroraKhera/blob/main/Phase_identification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import warnings
from copy import deepcopy
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import datetime
from scipy import stats

In [ ]:
#ConvertToPerUnit_Voltage#
def ConvertToPerUnit_Voltage(timeseries):
    ''' This function takes a voltage timeseries and converts it into a per
            unit representation.  This function looks at each customer's 
            timeseries individual mean, rounds the mean of the measurements and 
            compares that to a list of known base voltages.  Voltage levels may
            need to be added to that list as more voltages levels are used.  
            This allows for the case where some customers run at 240V and some
            run at 120V in the same dataset. The function will print a warning 
            if some customers have all NaN values, but it will complete 
            successfully in that case with that customer having all NaN values
            in the per-unit timeseries as well.  Supported base voltages are
            120, 240, 7200.  

        Parameters
        ----------
            timeseries: numpy array of float (measurements,customers) - the 
                raw AMI voltage measurements

        Returns:
            voltagePU: numpy array of float (measurements,customers) -  the 
                voltage timeseries converted into per-unit representation
                
        '''
    
    voltageMismatchThresh = .8
    voltageLevels = np.array([120,240,7200])
    voltagePU = np.zeros((timeseries.shape),dtype=float)
    dataLength = timeseries.shape[0]
    
    for custCtr in range(0, timeseries.shape[1]):
        currentCust = timeseries[:,custCtr]
        # Check for the case a customer only has NaN values in the timeseries
        if np.sum(np.isnan(currentCust)) == dataLength:
            print('Warning!  Customer index ' + str(custCtr) + ' only had NaN values in the timeseries. ')
            voltagePU[:,custCtr] = currentCust
            continue
        else:
            meanValue = np.round(np.nanmean(currentCust),decimals=0)
            vDiff = np.abs(voltageLevels - meanValue)
            index = np.argmin(vDiff)
            
            if index == 0:
                print('Customer index ' + str(custCtr) + ' is a 120V customer')
            # Check for the case where the correct voltage level is not listed
            if np.abs(vDiff[index]) > (voltageMismatchThresh*voltageLevels[index]):
                print('Error!  Customer# ' + str(custCtr) + 'has a mean voltage value of ' + str(meanValue) + '.  This voltage level is not supported in the function.  Please add this voltage level to the source code of the function')
                return (-1)
            voltagePU[:,custCtr] = np.divide(currentCust, voltageLevels[index])
    return voltagePU
# End of ConvertToPerUnit_Voltage

In [ ]:
##############################################################################
#
#                           CalcDeltaVoltage
#
def CalcDeltaVoltage(voltage):
    ''' This function takes a voltage timeseries and takes the difference
        between adjacent measurements, converting each measurement into a 
        change in voltage between timesteps.

        Parameters
        ----------
            timeseries: numpy array (measurements,customers), the voltage
                measurements

        Returns:
            deltaVoltage: numpy array (measurements,customers), the voltage 
                timeseries converted into the difference representation, ie
                the change in voltage at each timestep
        '''

    deltaVoltage = np.diff(voltage, n=1, axis=0)
    return deltaVoltage
# End of CalcDelta Voltage


In [ ]:
##############################################################################
#
# GetVoltWindow
# 
def GetVoltWindow(voltage,windowSize,windowCtr):
    """ This function takes the voltage time series and retrieves a particular
        window based on the windowCtr parameter
            
            Parameters
            ---------
                voltage: numpy array of float (measurements,customers) time 
                    series voltage measurements
                windowSize: int scalar representing the desired window size
                windowCtr: int scalar representing which window to use

                
                
            Returns
            -------
                voltBatch: numpy array of float (windowSize, customers)
                    one window of the voltage time series
            """
                    
    start = windowCtr * windowSize
    end = windowSize * (windowCtr + 1)
    voltBatch = voltage[start:end,:]
    return voltBatch
# End of GetVoltWindow

In [ ]:
##############################################################################
#
# CleanVoltWindow
#
def CleanVoltWindow(voltWindow,currentCustIDs,currentPhaseLabels):
    """ This function takes a window of voltage time series and removes customers
        which contain missing data during that window.  This function is for 
        the phase identification task and does not include transformer labels.
        Use CleanVoltWindowTrans for the transformer pairing task
            
            Parameters
            ---------
                voltWindow: numpy array of float (measurements,customers) time 
                    series voltage measurements
                currentCustIDs: numpy array of strings (customers) with the 
                    IDs of the customers currently in use
                currentPhaseLabels: numpy array of int (1,customers) with the
                    phase labels of the current customers in use

                
                
            Returns
            -------
                voltWindow: numpy array of float (measurements, customers) the
                    same volt window with the customers which had missing data
                    during the window removed
                currentCustIDs: numpy array of strings (customers) same list
                    of customers without the 'cleaned' customers
                currentPhaseLabels: numpy array of in (1,customers) same list
                    of phase labels without the 'cleaned customers
            """
                              
    badIndices = []
    for custCtr in range(0,voltWindow.shape[1]):
        temp = voltWindow[:,custCtr]
        indices = np.where(np.isnan(temp))
        if len(indices[0]) != 0:
            badIndices.append(custCtr)
    voltWindow = np.array(voltWindow)
    voltWindow = np.delete(voltWindow,badIndices,axis=1)
    currentCustIDs = np.delete(currentCustIDs,badIndices)
    currentPhaseLabels = np.delete(currentPhaseLabels,badIndices,axis=1)
    #voltWindow = pd.DataFrame(voltWindow)
    return voltWindow,currentCustIDs,currentPhaseLabels
# End of CleanVoltWindow


In [ ]:
##############################################################################
#
# CleanVoltWindowNoLabels
#
def CleanVoltWindowNoLabels(voltWindow,currentCustIDs):
    """ This function takes a window of voltage time series and removes customers
        which contain missing data during that window.  This function is for 
        the phase identification task and does not include transformer labels.
        Use CleanVoltWindowTrans for the transformer pairing task.  This version
        of the function does not include the utility phase labels
            
            Parameters
            ---------
                voltWindow: numpy array of float (measurements,customers) time 
                    series voltage measurements
                currentCustIDs: numpy array of strings (customers) with the 
                    IDs of the customers currently in use
            
            Returns
            -------
                voltWindow: numpy array of float (measurements, customers) the
                    same volt window with the customers which had missing data
                    during the window removed
                currentCustIDs: numpy array of strings (customers) same list
                    of customers without the customers which had missing data
                    during the window removed
            """
                              
    badIndices = []
    for custCtr in range(0,voltWindow.shape[1]):
        temp = voltWindow[:,custCtr]
        indices = np.where(np.isnan(temp))
        if len(indices[0]) != 0:
            badIndices.append(custCtr)
    voltWindow = np.array(voltWindow)
    voltWindow = np.delete(voltWindow,badIndices,axis=1)
    currentCustIDs = np.delete(currentCustIDs,badIndices)
    #voltWindow = pd.DataFrame(voltWindow)
    return voltWindow,currentCustIDs
# End of CleanVoltWindowNoLabels
                         

In [ ]:
##############################################################################
#
#       CreateAggWeightMatrix
#
def CreateAggWeightMatrix(custID):
    """ This function takes list of customer IDs and returns an empty (all zero)
        weight matrix for the phase identification case where existing phase
        labels are not used.  
            
            Parameters
            ---------
                custID: list of string containing the IDs of each customer
                
                
            Returns
            -------
                aggWM: ndarray of float the aggregated weight matrix initialized
                    to all zeros.  This will update with weights from each window,
                    tracking paired/unpaired customer information
            """
    aggWM = np.zeros((len(custID),len(custID)),dtype=float)
    return aggWM
# End of CreateAggWeightMatrix

In [ ]:
###############################################################################
#
# CalcCorrCoef
#
def CalcCorrCoef(voltageWin):
    ''' This function takes a voltage window, calculates the correlation
        coefficients, checks for failure of calculating good correlation
        coefficients (if the resulting matrix is not positive definite, the
        function returns and error), and returns the CC matrix.

        Parameters
        ----------
            voltageWin: numpy array of floats (customers, measurements), the
                window of voltage measurements

        Returns:
            corrCoef: numpy array of floats (customers, customers), the
                resulting correlation coefficient matrix, will be mirrored
                across the diagonal, with 1's on the diagonal
            failFlag: boolean - true if correlation coefficient matrix is
                not positive definite
        '''

    failFlag = 0
    voltageWin = np.array(voltageWin)
    voltageWin = voltageWin.transpose()
    warnings.simplefilter("error", RuntimeWarning)
    try:
        corrCoef = np.corrcoef(voltageWin)
    except RuntimeWarning:
        print('RuntimeWarning caught in CalCorrCoefDistances')
        failFlag = 1
        corrCoef = 0

    return corrCoef,failFlag
# End of CalcCorrCoef
                    

In [ ]:
##############################################################################
#
#       UpdateAggWM
#
def UpdateAggWM(clusterLabels,custID,currentIDs,aggWM,windowCtr):
    """ This function takes cluster labels resulting from the spectral clustering
        of a window, the existing weight matrix and the customer ids to update
        the weights based on the current window results.  Paired customers' 
        (based on the spectral clustering labels) weights are incremented. 
            
            Parameters
            ---------
                clusterLabels: ndarray of int representing the cluster labeling
                    of each customer from the spectral clustering algorithm
                custID: list of string containing the IDs of each customer
                currentCustID: list of strings containing the IDs of each customer
                    clustered in the current window (does not include customers
                    with missing data in this window)
                aggWM: ndarray of float, shape (customers,customers) the 
                    aggregated weight matrix previously initialized
                windowCtr: ndarray of int, shape (customers,customers) list containing 
                    a count of how many windows each customer was clustered in
            Returns
            -------
                aggWM: ndarray of float the aggregated weight matrix previously
                    initialized and updated with the new informaiton from this window.
                windowCtr: ndarray of int, shape (1,customers) list containing 
                    a count of how many windows each customer was clustered in
            """
    
    allIndices = []
    for custCtr in range(0,len(currentIDs)):
        custIDStr = np.array(custID,dtype=str)
        custIndex = np.where(currentIDs[custCtr]==custIDStr)[0][0]
        allIndices.append(custIndex)
        updateIndices = np.where(clusterLabels==clusterLabels[custCtr])[0]
        updateIndicesTrue = np.in1d(custIDStr,currentIDs[updateIndices])
        updateIndicesTrue = np.where(updateIndicesTrue==True)[0]
        aggWM[custIndex,updateIndicesTrue] = aggWM[custIndex,updateIndicesTrue] + 1
    if len(custID) == len(currentIDs):
        windowCtr = windowCtr + 1
    else:
        for custCtr in range(0,len(allIndices)):
            windowCtr[allIndices[custCtr],allIndices] = windowCtr[allIndices[custCtr],allIndices] + 1
    # End of custCtr for loop
    return aggWM, windowCtr
# End of UpdateAggWM function

In [ ]:
##############################################################################
#
#       NormalizeAggWM
#
def NormalizeAggWM(aggWM,windowCtr):
    """ This function takes the finished, aggregated weight matrix and divides
        each entry by the number of windows that customer was clustered in.  
        This removes the factor of some customers being clustered in fewer
        windows than other customers.
            
            Parameters
            ---------
                aggWM: ndarray of float, shape (customers,customers) the 
                    aggregated weight matrix previously initialized
                windowCtr: ndarray of int, shape (1,customers) list containing 
                    a count of how many windows each customer was clustered in
            Returns
            -------
                aggWM: ndarray of float the aggregated weight matrix previously
                    initialized and updated with the new informaiton from this window.
            """
    onesArray = np.ones((1,np.shape(windowCtr)[1]))
    for custCtr in range(0,aggWM.shape[0]):
        numberArray= onesArray * windowCtr[0,custCtr]
        aggWM[custCtr,:] = np.divide(aggWM[custCtr,:],numberArray)
    return aggWM
# End of NormalizeAggWM function


In [ ]:
################################################################################
#
#       CalcPredictedPhaseNoLabels
#
def CalcPredictedPhaseNoLabels(finalClusterLabels, clusteredPhaseLabelErrors,clusteredIDs):
    ''' This function takes the final cluster labels from an ensemble spectral 
        clustering run and the original utility labels and assigns a predicted
        phase to each customer based on the utility phase labels.  Note that
        the approach works well if you are reasonably confident in the original
        utility labeling, i.e. if 50% or more of the original phase labels are
        incorrect then these predicted phase will likely be innaccurate even 
        though the final cluster labels produced by the ensemble should still
        reflect the correct phase groups.
        
        Parameters
        ---------
            finalClusterLabels: ndarray, shape (customers) containing the final
                cluster labels representing the phase predictions.  But these 
                label numbers will not match the real phases
            clusteredPhaseLabelErrors: ndarray, shape (1,customers) containing 
                phase labels for the customers which were clustered.  The
                dimensions should match the length of finalClusterLabels.  This
                is the original phase labels and may contain errors in the 
                labeling
            clusteredIDs: ndarray of str, shape(customers) has the customers 
                which recieved a predicted phase

        Returns
        -------
            predictedPhases: ndarray of int, shape (1,customers) containing the 
                predicted phase labels based on the majority vote of the final
                clusters
        '''
    predictedPhases = np.zeros((1,clusteredIDs.shape[0]),dtype=int)
    # Assign a predicted (actual) phase to each cluster
    numberOfClusters = (np.unique(finalClusterLabels)).shape[0]
    uniqueClusters = np.unique(finalClusterLabels)
    for clustCtr in range(0,numberOfClusters):
        currentCluster = uniqueClusters[clustCtr]
        indices1 = np.where(finalClusterLabels==currentCluster)[0]     
        clusterPhases = clusteredPhaseLabelErrors[0,indices1]
        pPhase = stats.mode(clusterPhases)[0][0]
        predictedPhases[0,indices1] = pPhase        

    return predictedPhases
# End of  CalcPredictedPhaseNoLabels

In [ ]:

################################################################################
#
#       CalcAccuracyPredwGroundTruth
#
def CalcAccuracyPredwGroundTruth(predictedPhases, clusteredPhaseLabels,clusteredIDs):
    ''' This function takes the predicted phase labels, the ground truth labels
        and the list of clustered customers to calculate accuracy.
        
        Parameters
        ---------
            predictedPhases: ndarray of int, shape (1,customers) containing the 
                predicted phase labels based on the majority vote of the final
                clusters
            clusteredPhaseLabels: numpy array of int (1,customers) - the 
                ground truth phase labels for each customer who received a 
                predicted phase.  This dimensions of this should match 
                predictedPhases
            clusteredIDs: ndarray of str, shape(customers) has the customer IDs 
                for customers which recieved a predicted phase

        Returns
        -------
            accuracy: float, decimal accuracy 
            incorrectCustCount: int, number of customers incorrectly classified
        '''
        
    incorrectCustCount = 0
    for custCtr in range(0,len(clusteredIDs)):
        #index = np.where(clusteredIDs[custCtr]==custIDStr)[0][0]
        if predictedPhases[0,custCtr] != clusteredPhaseLabels[0,custCtr]:
            incorrectCustCount = incorrectCustCount + 1
    numNotClust = clusteredPhaseLabels.shape[1] - len(clusteredIDs)
    accuracy = (clusteredPhaseLabels.shape[1]-(incorrectCustCount+numNotClust)) / clusteredPhaseLabels.shape[1]
    return accuracy, incorrectCustCount
# End of CalcAccuracyPredwGroundTruth

In [ ]:
##############################################################################
#
#       UpdateCustWindowCounts
#
def UpdateCustWindowCounts(custWindowCounts,currentIDs,custIDInput):
    """ This function updates the number of windows that each customer was 
            included in the analysis, i.e. not excluded due to missing data. 
            This function assumes that all entries in currentIDs are contained
            in the list of IDs in custIDInput and does not do error check for
            that fact.
            
            Parameters
            ---------
                custWindowCounts: numpy array of int (total number of customers)- the
                    window count for each customer
                currentIDs: list of str (current number of customers) - the list of 
                    customers IDs included in the current window to be added
                    to the custWindowCounts
                custIDInput: list of str (total number of customers) - the 
                    complete list of customer IDs. The length of this should
                    match the length/indexing of custWindowCounts
            Returns
            -------
                custWindowCounts: numpy array of int (total number of customers)
                    the window count for each customer that has been updated
                    with the new information in currentIDs
            """
    
    for custCtr in range(0,len(currentIDs)):
        index = np.where(np.array(custIDInput)==currentIDs[custCtr])[0][0]
        custWindowCounts[index] = custWindowCounts[index] + 1
    return custWindowCounts
# End of UpdateCustWindowCounts function
                    

In [ ]:
#############################################################################
#
#                           DropCCUsingLowCCSep
#
def DropCCUsingLowCCSep(ccMatrixInput,lowCCSepThresh,sensIDInput):
    """ This function takes the correlation coefficient results from a single window, 
        assuming that the window is from a mix of customers to sensors and removes any
        CC that have a lower CC Separation Score than the specified threshold
            
            Parameters
            ---------
                ccMatrixInput: numpy array of float (customer,sensors)
                    the CC between customers and sensors for a single window
                lowCCSepThresh: float - the CC Separation threshold, any
                    CC values with CC Separation lower than this are discarded
                sensIDInput: list of str - the list of sensor IDs
                
            Returns
            -------
                ccMatrixAdjusted: numpy array of float (customers,sensors) -
                    The CC Matrix with CC values with CC Separation less than
                    the threshold discarded

            """                
        
    ccMatrixAdjusted = deepcopy(ccMatrixInput)
    sensUnique = np.unique(sensIDInput)
    for custCtr in range(0,ccMatrixAdjusted.shape[0]):
        #### Calculate the ccSeparation section
        currCC = ccMatrixAdjusted[custCtr,:]
        for sensCtr in range(0,len(sensUnique)):
            currSensor = sensUnique[sensCtr]
            indices = np.where(np.array(sensIDInput)==currSensor)[0]
            ccSet = np.sort(currCC[indices])
            ccDiff = ccSet[-1] - ccSet[-2]            
            if ccDiff < lowCCSepThresh:
                ccMatrixAdjusted[custCtr,indices] = 0
    return ccMatrixAdjusted
# End of DropCCUsingLowCCSep



In [ ]:

##############################################################################
#
# FilterPredictedCustomersByConf
# 
def FilterPredictedCustomersByConf(custIDPredInc,custIDInput,newPhaseLabels,orgDiffPhaseLabels,winVotesConfScore=-1,
                                   ccSeparation=-1,sensVotesConfScore=-1,combConfScore=-1,winVotesThresh=-1, 
                                   ccSepThresh=-1,sensVotesThresh=-1,combConfThresh=-1):
    """ This function takes a list of customers predicted to have incorrect 
            phases and filters them by confidence scores using provided 
            thresholds.  Any provided confidence scores and threshold provided
            are used, otherwise they are ignored
            
            Parameters
            ---------
                custIDPredInc: list of str - the list of customer IDs that 
                    we are predicted to have incorrect phase labels
                custIDInput: list of str - the full list of customer IDS
                newPhaseLabels: numpy array of int (1,customers) - the 
                    predicted phase labels for the customers in custIDPredInc
                orgDiffPhaseLabels: numpy array of int (1,customers) - the 
                    original phase labels for the customers in custIDPredInc
                winVotesConfScore: list of float - the window voting confidence
                    score for each customer. 
                ccSeparation: list of float - the correlation coefficient 
                    separation for each customer
                sensVotesConfScore: list of float - the sensor agreement
                    score for each customer
                combConfScore: list of float - the combined confidence score
                    for each customer
                winVotesThresh: float - the threshold to use for the window
                    voting score
                ccSepThresh: float - the threshold to use for the CC separation
                    score
                sensVotesThresh: float - the threshold to use for the 
                    sensor agreement score
                combConfThresh: float - the threshold to use for the combined
                    confidence score
            Returns
            -------
                filteredCustIDPredInc: list of str - the custIDPredInc list
                    filtered by confidence scores.  The remaining customers
                    will have confidence scores above the specified thresholds
                filteredNewPhaseLabels: numpy array of int (1,customers) - the
                    predicted phase labels for the customers in filteredCustIDPredInc
                filteredOrgPhaseLabels: numpy array of int (1,customers) - the
                    original phase labels for the customers in filteredCustIDPredInc
                    
            """
                    
    deleteList = set({})
    for custCtr in range(0, len(custIDPredInc)):
        currIndex = custIDInput.index(custIDPredInc[custCtr])
        
        if type(winVotesThresh) != int:
            if winVotesConfScore[currIndex] < winVotesThresh:
                deleteList.add(custCtr)
        if type(ccSepThresh) != int:
            if ccSeparation[currIndex] < ccSepThresh:
                deleteList.add(custCtr)
        if type(sensVotesThresh) != int:
            if sensVotesConfScore[currIndex] < sensVotesThresh:
                deleteList.add(custCtr)
        if type(combConfThresh) != int:
            if combConfScore[currIndex] < combConfThresh:
                deleteList.add(custCtr)
    deleteList = list(deleteList)
    filteredCustIDPredInc = list(np.delete(np.array(custIDPredInc),deleteList))
    filteredNewPhaseLabels = np.delete(newPhaseLabels,deleteList)
    filteredOrgPhaseLabels = np.delete(orgDiffPhaseLabels,deleteList)
    return filteredCustIDPredInc, filteredNewPhaseLabels,filteredOrgPhaseLabels
# End of FilterPredictedCustomersByConf


In [ ]:
#############################################################################
#
#                           CountClusterSizes
#
def CountClusterSizes(clusterLabels):
    """ This function takes the labels produced by spectral clustering (or
        other clustering algorithm) and counts the members in each cluster.  
        This is primarily to see the distribution of cluster sizes over all
        windows, particularly to see if there singleton clusters or a significant
        number of clusters with a small number of members.
            
            Parameters
            ---------
                clusterLabels: numpy array of int (clustered customers) - the cluster 
                    label of each customer
                
            Returns
            -------
                clusterCounts: numpy array of int (0,k) - the number of customers
                    in each cluster
            """                
        
    currentK = len(np.unique(clusterLabels))
    clusterCounts = np.zeros((1,currentK),dtype=int)
    for clustCtr in range(0,currentK):
        indices = np.where(clusterLabels==clustCtr)[0]
        clusterCounts[0,clustCtr] = len(indices)
    return clusterCounts
# End of CountClusterSizes


In [ ]:
# Start of the PlotHistogramOfWinVotesConfScore function.
def PlotHistogramOfWinVotesConfScore(winVotesConfScore,savePath=-1):
    """ This function takes the list of the window votes score which is a confidence
            score based on the percentage of window votes which were the same
            for each customer, and plots a histogram.  For example, if each
            window were taken individually (not in an ensemble) and a predicted
            phase was assigned for each customer in each window, the window 
            votes confidence score for a particular customer would be the percentage
            of windows which agree on the phase.  
            
    Parameters
    ---------
        winVotesConfScore: list of float - the list containing the decimal
            confidence score defined as the mode of the phase votes over all
            windows divided by the total number of windows
        savePath: str or pathlib object - the path to save the histogram 
            figure.  If none is specified the figure is saved in the current
            directory
                
    Returns
    -------
        None
                
            """
    
    plt.figure(figsize=(12,9))
    sns.histplot(winVotesConfScore)
    plt.xlabel('Window Votes Confidence Score', fontweight = 'bold',fontsize=32)
    plt.ylabel('Count', fontweight = 'bold',fontsize=32)
    plt.yticks(fontweight='bold',fontsize=20)
    plt.xticks(fontweight='bold',fontsize=20)
    plt.title('Histogram of Window Votes Confidence Score',fontweight='bold',fontsize=12)
    plt.show()
    plt.tight_layout()    
    today = datetime.datetime.now()
    timeStr = today.strftime("%Y-%m-%d_%H-%M-%S")
    filename = 'WinVotesConfScore_HIST'
    filename = filename + timeStr + '.png'
    # Save Figure
    if type(savePath) is int:   
        plt.savefig(filename)
    else:
        plt.savefig(Path(savePath,filename))  
#End of PlotHistogramOfWinVotesConfScore

In [ ]:

# Start of the PlotHistogramOfCombinedConfScore function.
def PlotHistogramOfCombinedConfScore(confScoreCombined,savePath=-1):
    """ This function takes the list of the score which is the combination 
            (multiplied together) of the window voting score and the 
            sensor agreement confidence socre and plots a histogram
            
    Parameters
    ---------
        confScoreCombined: list of float - the list containing combination of
            the winVotesConfScore and the sensVotesConfScore by multiplying
            them together
        savePath: str or pathlib object - the path to save the histogram 
            figure.  If none is specified the figure is saved in the current
            directory
                
    Returns
    -------
        None
                
            """
    
    plt.figure(figsize=(12,9))
    sns.histplot(confScoreCombined)
    plt.xlabel('Combined Confidence Score', fontweight = 'bold',fontsize=32)
    plt.ylabel('Count', fontweight = 'bold',fontsize=32)
    plt.yticks(fontweight='bold',fontsize=20)
    plt.xticks(fontweight='bold',fontsize=20)
    plt.title('Histogram of Combined Confidence Score',fontweight='bold',fontsize=12)
    plt.show()
    plt.tight_layout()    
    today = datetime.datetime.now()
    timeStr = today.strftime("%Y-%m-%d_%H-%M-%S")
    filename = 'CombinedConfScore_HIST'
    filename = filename + timeStr + '.png'
    # Save Figure
    if type(savePath) is int:   
        plt.savefig(filename)
    else:
        plt.savefig(Path(savePath,filename))  
#End of PlotHistogramOfCombinedConfScore

In [ ]:
# Start of the PlotHistogramOfSensVotesConfScore function.
def PlotHistogramOfSensVotesConfScore(sensVotesConfScore,savePath=-1):
    """ This function takes the list of the scores representing the percentage
            of sensors which agreed in the phase prediction for each customer
            
    Parameters
    ---------
        sensVotesConfScore: list of float - the list containing the decimal 
            value for the percentage of sensors which agreed in the phase
            prediction for each customers.  This will be 1 if all sensors 
            agree on the prediction
        savePath: str or pathlib object - the path to save the histogram 
            figure.  If none is specified the figure is saved in the current
            directory
                
    Returns
    -------
        None
                
            """
    
    percentages = np.array(sensVotesConfScore) * 100
    plt.figure(figsize=(12,9))
    sns.histplot(percentages)
    plt.xlabel('Percentage of Sensor Agreement', fontweight = 'bold',fontsize=32)
    plt.ylabel('Count', fontweight = 'bold',fontsize=32)
    plt.yticks(fontweight='bold',fontsize=20)
    plt.xticks(fontweight='bold',fontsize=20)
    plt.title('Histogram of Sensor Agreement',fontweight='bold',fontsize=12)
    plt.show()
    plt.tight_layout()    
    today = datetime.datetime.now()
    timeStr = today.strftime("%Y-%m-%d_%H-%M-%S")
    filename = 'SensorAgreementConfScore_HIST'
    filename = filename + timeStr + '.png'
    # Save Figure
    if type(savePath) is int:   
        plt.savefig(filename)
    else:
        plt.savefig(Path(savePath,filename))  
#End of PlotHistogramOfSensVotesConfScore
    

In [ ]:

# Start of the PlotHistogramOfCCSeparation function.
def PlotHistogramOfCCSeparation(ccSeparation,xLim=0.2,savePath=-1):
    """ This function takes the list of the correlation coefficient separation
            confidence scores and plots a histogram.  
            
    Parameters
    ---------
        ccSeparation: list of float - the list containing the separation 
            between the labeled phase CC and the next highest CC for each 
            customer.  This is used as a type of confidence score
        xLim: float - the value for the x-axis limit for the figure.  0.2 is
            the default because that works well with the our utility data. If -1 is 
            used, the function will not specify an x-axis limit.
        savePath: str or pathlib object - the path to save the histogram 
            figure.  If none is specified the figure is saved in the current
            directory
                
    Returns
    -------
        None
                
            """
            
    plt.figure(figsize=(12,9))
    sns.histplot(ccSeparation)
    plt.xlabel('Correlation Coefficient Separation', fontweight = 'bold',fontsize=32)
    plt.ylabel('Count', fontweight = 'bold',fontsize=32)
    plt.yticks(fontweight='bold',fontsize=20)
    plt.xticks(fontweight='bold',fontsize=20)
    if xLim != -1:
        plt.xlim(0,xlim=xLim)
    plt.title('Histogram of Correlation Coefficient Separation',fontweight='bold',fontsize=12)
    plt.show()
    plt.tight_layout()    
    today = datetime.datetime.now()
    timeStr = today.strftime("%Y-%m-%d_%H-%M-%S")
    filename = 'CCSeparation_HIST'
    filename = filename + timeStr + '.png'
    # Save Figure
    if type(savePath) is int:   
        plt.savefig(filename)
    else:
        plt.savefig(Path(savePath,filename))
    plt.close()
#End of PlotHistogramOfCCSeparation

##the functions that implement the Co-Association Matrix Ensemble Method for the phase identification task.

    Functions:
        -  SPClustering
        -  SPClustering_Precomp
        -  CAEnsemble
        

In [ ]:
from sklearn.cluster import SpectralClustering
import numpy as np
from copy import deepcopy
# Import - Custom Libraries
import PhaseIdent_Utils as PIUtils

In [ ]:
###############################################################################
#
#                                  SPClustering
#

def SPClustering(features,k):
    """ This function takes a window of timeseries data for the total number of
         customers and the number of desired clusters and performs the spectral 
         clustering algorithm on that data, returning the cluster labels for each 
         customer.  This is the internal spectral clustering function which is
         called for each window (and each value in kVector).  These results
         are used to build the co-association matrix.  
         The kernel function has been hardcoded here to be the Radial
         Basis Function ('rbf') based on the results of this research.
            
            Parameters
            ---------
                features: numpy array of float (customers,measurments) - a 
                    'window' of time series measurements where customers with
                    missing data are removed.  Any NaN values in this matrix
                    will cause the SpectralClustering function to fail.  
                k:  int - Number of clusters
                
            Returns
            -------
                clusterLabels:  list of int - The resulting cluster label of 
                    each customer (1-k)
            """       
    
    sp = SpectralClustering(n_clusters=k,affinity='rbf')
    clusterLabels = sp.fit_predict(features)    
    return clusterLabels       
# End of SPClustering

In [ ]:
###############################################################################
#
#                                  SPClustering_Precomp
#

def SPClustering_Precomp(aggWM,kFinal):
    """ This function takes a precomputed affinity matrix, in the form
        of a co-association matrix generated by CAEnsemble and will
        use that to construct the final clusters representing the three-phases.

            Parameters
            ---------
                aggWM: ndarray of float, shape (customers,customers) affinity 
                    matrix of paired/unpaired weights aggregated over all 
                    available windows.
                kFinal: int - the number of final clusters. This parameter 
                    should be set based on the feeder topology.  Setting this
                    parameter to 4 of 7 is a good place to start.  If the feeder
                    in question has voltage regulating devices a larger number 
                    of final clusters may be required.
                
            Returns
            -------
                clusterLabels:  list of int - The resulting cluster label of 
                    each customer (1-k)
            """       

    sp = SpectralClustering(n_clusters=kFinal,n_init=10,assign_labels='discretize',affinity='precomputed')
    clusterLabels = sp.fit_predict(aggWM)    
    return clusterLabels       
# End of SPClustering_Precomp

In [ ]:
###############################################################################
#
#                       CAEnsemble
#

def CAEnsemble(voltage,kVector,kFinal,custID,windowSize,lowWindowsThresh=4,printLowWinWarningFlag=True):

    """ This function implements the ensemble of Spectral Clustering  for the
        task of phase identification task.  The ensemble size is determined by 
        the number of sliding windows available given the windowSize parameter.
        In each window the cluster labels are returned by the spectral clustering
        algorithm and that clustering is then used to update a co-association matrix
        based on pairwise paired/unpaired information in the cluster labels.  
        That weight matrix is then used for a final clustering into the final
        clusters which represent phase groupings.  The original utility phase
        labels are not used in this function.  The mapping of the final clusters
        to particular phases is left to a subsequent step.  
        For more details, please see this paper:  
        L. Blakely and M. J. Reno, “Phase Identification Using Co-Association Matrix Ensemble Clustering,” IET Smart Grid, no. Machine Learning Special Issue, Jun. 2020.
            
            Parameters
            ---------
                voltage:  numpy array of float (measurements,customers) -  
                    voltage timeseries for each customer.  The timeseries 
                    should be pre-processed into per-unit, difference (delta)
                    representation.  This pre-processing is an essential step.
                kVector: numpy array of int - a vector of the possible values of
                    k for the windows
                kFinal:  int - Number of clusters for the final clustering
                custID: list of str - list of customer ids
                windowSize:  int - The size (in number of measurements) of the 
                    sliding window
                lowWindowsThresh: int - the minimum number of windows before
                    printing a warning that some customers had few windows 
                    due to missing data.  The default value is set to 4 if 
                    this parameter is not specified.
                printLowWinWarningFlag: boolean - allows supression of the printout
                    if customer has only a few windows in the ensemble.  The
                    default value is True.  If a customer is only present in
                    a small number of windows the co-association matrix will not
                    be built adequately for that customer (although it will not
                    affect other customers).  Thus results for customers with
                    few windows should be considered low confidence predictions
                    and likely discarded
                
            Returns
            -------
                finalClusterLabels:  numpy array of int (1,customers) 
                    array of the final cluster labels representing the 
                    phases, but they will not match in number to the actual phases
                    Determining which cluster number goes with which real phase
                    is left for a future step.  This parameter is one that 
                    depends on the topology of the feeder.  For more discussion
                    see the paper by B.D. Pena listed above.  Starting values
                    to try for this parameter might be 4 or 7, topologies with
                    voltage regulators in the feeder may require a larger 
                    number of final clusters.  
                noVotesIndex:  list of int - list of customer indices that 
                    did not recieve any votes (i.e. were removed from all 
                    windows).  This occurs due to missing data for a customer.
                    If all windows for that customer contain missing data, then
                    that customer will be eliminated from the analysis.
                noVotesIDs:  list of str - list of customer ids that did not 
                    receive any votes (i.e. were removed from all windows due 
                    to missing data)
                clusteredIDs:  list of str (customers) - list of customers IDs 
                    that were clustered during the ensemble.  The length of 
                    clusteredIDs plus the length of noVotesIDs should equal
                    the total number of customers
                custWindowCounts: numpy array of int (customers) - the count,
                    for each customer, of the number of windows that were
                    included in the analysis, i.e. the number of windows that 
                    were not excluded due to missing data.  This count is 
                    significantly affected by the value chosen for the 
                    windowSize parameter.  Customers with a low number of 
                    windows in the ensemble should be considered low confidence
                    in the final prediction as they will not populate the 
                    co-association matrix properly.  
            """       
    
    ensTotal = int(np.floor(voltage.shape[0] / windowSize))  # This determines the total number of windows based on available data and window size
    ensPredictedPhases = np.zeros((1,len(custID)),dtype=int)
    aggWM = PIUtils.CreateAggWeightMatrix(custID) # This is the co-assocation matrix
    windowCtr = PIUtils.CreateAggWeightMatrix(custID) # This tracks the number of windows where each pair of customers was included together
    allClusterCounts = []
    custWindowCounts = np.zeros((len(custID)),dtype=int) # This tracks the number of windows used for each customer

    # Loop through each window in the available data
    for ensCtr in range(0,ensTotal):
        print('ensCtr = ' + str(ensCtr))
        #Select the next time series window and remove customers with missing data in that window
        windowDistances = PIUtils.GetVoltWindow(voltage,windowSize,ensCtr)
        currentDistances,currentIDs = PIUtils.CleanVoltWindowNoLabels(deepcopy(windowDistances), deepcopy(custID))
        custWindowCounts = PIUtils.UpdateCustWindowCounts(custWindowCounts,currentIDs,custID)
       
        # Check for the case where the entire distance matrix is nans
        if ((currentDistances.shape[0] == 1) and (currentDistances.shape[1] == 1)):
            continue
        currentDistances = currentDistances.transpose()
        
        # Loop through each value of k (number of clusters) to use multiple numbers of clusters in each available window
        for kCtr in range(0,len(kVector)):
            k = kVector[kCtr]
            #Check if the cleaning reduced the number of available customers to less than the number of clusters
            if (currentDistances.shape[0] <= k):
                 continue
            #Do the clustering
            clusterLabels = SPClustering(currentDistances,k)
            #Update the weight matrix
            aggWM, windowCtr = PIUtils.UpdateAggWM(clusterLabels,custID,currentIDs,aggWM,windowCtr)  
            #Update Cluster Sizes List
            clusterCounts=np.squeeze(PIUtils.CountClusterSizes(clusterLabels))
            for kCtr2 in range(0,k):
                allClusterCounts.append(clusterCounts[kCtr2])
        #End of kCtr for loop
    # End of ensCtr for loop
    
    #Split customers into customers who had at least one window of data and those that did not
    # If a customer had missing data in all windows then they are not included in the algorithm results
    noVotesIndex = []
    noVotesIDs = []
    for custCtr in range(0,len(custID)):
        if (np.sum(aggWM[custCtr,:]) == 0):
            noVotesIndex.append(custCtr)
            noVotesIDs.append(custID[custCtr])
    clusteredIDs = np.delete(custID,noVotesIndex)
    ensPredictedPhases = np.delete(ensPredictedPhases,noVotesIndex,axis=1)
    aggWM = np.delete(aggWM,noVotesIndex,axis=0)
    aggWM = np.delete(aggWM,noVotesIndex,axis=1)
    windowCtr = np.delete(windowCtr,noVotesIndex,axis=0)
    windowCtr = np.delete(windowCtr,noVotesIndex,axis=1)
    
    if aggWM.shape == (0,0):
        print('Error!  All customers were eliminated from all windows, and the algorithm could not continue.  This is due to missing data in the customers datastreams.  The distribution of missing data was such that there were instances of missing data in every window for every customer.  You could try reducing the window size, but beware that there still may not be many viable windows ')
        return (-1,-1,-1,-1,-1)
    #Normalize aggWM - This is done because each customer would have had different numbers of windows due to missing data, the normalization is done by dividing each cell by the number of windows that pair of customers was both present
    windowCtr[windowCtr==0]=0.0001 #This prevents divide by zero, the aggWM should already be zero in the locations where windowCtr is 0, so 0 will be the end result in that case anway
    aggWM_Norm = np.divide(aggWM,windowCtr)
    aggWM_Norm[np.isnan(aggWM_Norm)] = 0
    aggWM_Norm[aggWM_Norm==0]=0.00001 # The spectral clustering function does not allow zeros (the precomputed matrix must be fully-connected), so any zeros are set to a very small value
    finalClusterLabels = SPClustering_Precomp(aggWM_Norm,kFinal)
    
    # Few windows warning
    numLowWindows = np.where(custWindowCounts <= lowWindowsThresh)[0]
    if printLowWinWarningFlag:
        if len(numLowWindows) != 0:
            print('Warning!  ' + str(len(numLowWindows)) + ' customers had fewer than ' + str(lowWindowsThresh) + ' windows used in the phase identification ensemble.  The predictions for these customers should likely be considered low confidence predictions')
            print('Customer IDs for customers with fewer than ' + str(lowWindowsThresh) + ' windows:')
            for custCtr in range(0,len(numLowWindows)):
                print('Customer ID: ' + str(custID[numLowWindows[custCtr]]) + ' - ' + str(custWindowCounts[numLowWindows[custCtr]]) + ' Windows')
            print('')
        
    return finalClusterLabels,noVotesIndex,noVotesIDs,clusteredIDs,aggWM_Norm,custWindowCounts
# End of CAEnsemble

**sample script for running the Co-Association Matrix Ensemble Phase Identfication algorithm which uses customer advanced metering 
infrastructure (AMI) data with an ensemble spectral clustering algorithm and 
co-association matrix to cluster customers by their service phase. 
This script requires above functions - CA_Ensemble_Funcs and PhaseIdent_Utils**

In [ ]:
"''Input Data Formatting:
    voltageInput: numpy array of float - This matrix contains the voltage 
        measurements (in volts) for all customers under consideration.  The
        matrix should be in the form (measurements,customers) where each 
        column represents one customer AMI timeseries.  It is recommended that
        the timeseries interval be at least 15-minute sampling, although the
        algorithm will still function using 30-minute or 60-minute interval
        sampling
    custIDInput: list of str - This is a list of customer IDs as strings.  The
        length of this list should match the customer dimension of voltageInput
    phaseLabelsErrors: numpy array of int - This contains the phase labels for
        each customer in integer form (i.e. 1 - Phase A, 2 - Phase B,
        3 - Phase C).  Any integer notation may be used for this field; it is 
        only used to assigned the final phase predictions.  In practice, this
        field could even be omitted, the phase identification results from 
        CAEnsemble will still be grouped by phase, and the assignment of final
        phase labels could be left for a post-processing step by the utility.
        The dimensions of this matrix should be (1, customers).
        These are assumed to be the original, utility labels, which
        may contain some number of errors.  The sample data included with these
        scripts has ~9% of phase labels injected with errors.  This can be 
        seen by comparing this field with the entries in phaseLabelsTrue which
        contains the ground-truth phase labels
    phaseLabelsTrue: numpy array of int (1,customers) - This contains the 
        ground-truth phase labels for each customer, if available.  Note that,
        in practice this may not be available, but for testing purposes this 
        is provided along with functions to evaluate the phase identifcation
        accuracy against the ground-truth labels.
        
The indexing of each of the input data must match, i.e. voltageInput[:,1] must
    represent the same customer as custIDInput[1], phaseLabelErrors[0,1] and
    phaseLabelsTrue[0,1]

"""

In [ ]:
import numpy as np
from pathlib import Path
import pandas as pd
# Import - Custom Libraries
import CA_Ensemble_Funcs as CAE
import PhaseIdent_Utils as PIUtils

In [ ]:
##############################################################################
#                    Load Sample Data

currentDirectory = Path.cwd()
filePath = Path(currentDirectory.parent,'SampleData')

filename = Path(filePath,'VoltageData_AMI.npy')
voltageInputCust = np.load(filename)
filename = Path(filePath,'PhaseLabelsTrue_AMI.npy')
phaseLabelsTrue = np.load(filename)
filename = Path(filePath,'PhaseLabelsErrors_AMI.npy')
phaseLabelsErrors = np.load(filename)
filename = Path(filePath,'CustomerIDs_AMI.npy')
custIDInput = list(np.load(filename))


In [ ]:
##############################################################################
###############################################################################
#
#
#         Co-Association Matrix Ensemble Phase Identification
#                   
#


# Data pre-processing steps
# This converts the original voltage timeseries (assumed to be in volts) into per-unit representation
vNorm = PIUtils.ConvertToPerUnit_Voltage(voltageInputCust)
# This takes the difference between adjacent measurements, converting the timeseries into a per-unit, change in voltage timeseries
vNDV = PIUtils.CalcDeltaVoltage(vNorm)


# kFinal is the number of final clusters produced by the algorithm.  Each 
#   cluster will represent a phase grouping of customers.  Ideally, this value
#   could be 3, however in practice usually a larger value is required.  Issues
#   such as customers located on adjancent feeders present in the data, voltage
#   regulators, or other topology issues may require tuning of this parameter.
#   This could be done using silhouette score analysis.  7 is likely a good
#   place to start with this parameter.
kFinal=7 

# kVector is the number of clusters used internally by the algorithm in each 
#   window.  
kVector =[6,12,15,30]
# windowSize is the number of datapoints used in each window of the ensemble
windowSize = 384

# This is the primary phase identification function - See documentation in CA_Ensemble_Funcs.py for details on the inputs/outputs
finalClusterLabels,noVotesIndex,noVotesIDs,clusteredIDs,aggWM,custWindowCounts = CAE.CAEnsemble(vNDV,kVector,kFinal,custIDInput,windowSize)

# Remove any omitted customers from the list of phase labels
if len(noVotesIndex) != 0:
    clusteredPhaseLabels = np.delete(phaseLabelsErrors,noVotesIndex,axis=1)
    clusteredTruePhaseLabels = np.delete(phaseLabelsTrue,noVotesIndex,axis=1)
    custIDFound = list(np.delete(np.array(custIDInput),noVotesIndex))
else:
    clusteredPhaseLabels = phaseLabelsErrors
    clusteredTruePhaseLabels = phaseLabelsTrue
    custIDFound = custIDInput
    
# Use the phase labels to assign final phase predictions based on the majority vote in the final clusters
# This assumes that phase labels are both available and believed to be reasonably accurate.
# In the case where phase labels are unavailable or believed to be highly innacurate, some other method of final phase prediction must be used.
predictedPhases = PIUtils.CalcPredictedPhaseNoLabels(finalClusterLabels, clusteredPhaseLabels,clusteredIDs)

# This shows how many of the predicted phase labels are different from the original phase labels
diffIndices = np.where(predictedPhases!=clusteredPhaseLabels)[1]

# If the ground-truth labels are available, this will calculate a true accuracy
accuracy, incorrectCustCount = PIUtils.CalcAccuracyPredwGroundTruth(predictedPhases, clusteredTruePhaseLabels,clusteredIDs)
accuracy = accuracy*100


print('Spectral Clustering Ensemble Phase Identification Results')
print('There are ' + str(diffIndices.shape[0]) + ' customers with different phase labels compared to the original phase labeling.')
print('')

print('The accuracy of the predicted phase is ' + str(accuracy) + '% after comparing to the ground truth phase labels')
print('There are '+ str(incorrectCustCount) + ' incorrectly predicted customers')
print('There are ' + str(len(noVotesIndex)) + ' customers not predicted due to missing data')



# Write outputs to csv file
df = pd.DataFrame()
df['customer ID'] = custIDFound
df['Original Phase Labels (with errors)'] = clusteredPhaseLabels[0,:]
df['Predicted Phase Labels'] = predictedPhases[0,:]
df['Actual Phase Labels'] = clusteredTruePhaseLabels[0,:]
df.to_csv('outputs_CAEnsMethod.csv')
print('Predicted phase labels written to outputs_CAEnsMethod.csv')



